In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, col, count, collect_list, explode, array_contains, size

# Create Spark session

In [2]:
# spark = SparkSession.builder \
#     .appName("University Information System") \
#     .config("spark.mongodb.input.uri", "mongodb://localhost:27017/university_information_system") \
#     .config("spark.mongodb.output.uri", "mongodb://localhost:27017/university_information_system") \
#     .getOrCreate()
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("University Information System") \
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
#     .config("spark.mongodb.input.uri", "mongodb://localhost:27017/university_information_system") \
#     .config("spark.mongodb.output.uri", "mongodb://localhost:27017/university_information_system") \
#     .getOrCreate()


spark = SparkSession.builder \
    .appName("University Information System") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/university_information_system") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/university_information_system") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()


24/09/22 15:50:00 WARN Utils: Your hostname, Parass-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.29.187 instead (on interface en0)
24/09/22 15:50:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/parasdhiman/.ivy2/cache
The jars for the packages stored in: /Users/parasdhiman/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7b35f10d-6062-4548-afa4-553fb480e72a;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


:: resolution report :: resolve 88ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-7b35f10d-6062-4548-afa4-553fb480e72a
	confs: [default]
	0 artifacts copied, 4 already retrieved (0kB/4ms)
24/09/22 15:50:00 WARN NativeCodeLoader: Unable to load native-hadoop library f

# Load collections into DataFrames

In [3]:
students_df = spark.read.format("mongo").option("collection", "students").load()
courses_df = spark.read.format("mongo").option("collection", "courses").load()
instructors_df = spark.read.format("mongo").option("collection", "instructors").load()
departments_df = spark.read.format("mongo").option("collection", "departments").load()

In [4]:
students_df.show()
courses_df.show()
instructors_df.show()
departments_df.show()

+---+-------------+--------------------+--------------------+----------+---------+
|_id|department_id|               email|         enrollments|first_name|last_name|
+---+-------------+--------------------+--------------------+----------+---------+
|  1|            1|john.doe@example.com|[{1, A }, {2, B }...|      John|      Doe|
|  2|            2|jane.smith@exampl...|  [{1, A }, {4, B }]|      Jane|    Smith|
|  3|            3|alice.johnson@exa...|  [{3, B }, {5, C }]|     Alice|  Johnson|
|  4|            4|bob.brown@example...|           [{4, C }]|       Bob|    Brown|
|  5|            5|charlie.davis@exa...|  [{5, B }, {6, A }]|   Charlie|    Davis|
|  6|            6|eva.white@example...|           [{6, A }]|       Eva|    White|
|  7|            7|liam.wilson@examp...| [{7, B }, {22, A }]|      Liam|   Wilson|
|  8|            8|sophia.martinez@e...| [{8, B }, {23, C }]|    Sophia| Martinez|
|  9|            9|noah.anderson@exa...|           [{9, A }]|      Noah| Anderson|
| 10

In [5]:
if spark.sparkContext is None:
    print("SparkContext is not active!")
else:
    print("SparkContext is active.")


SparkContext is active.


# 1. Fetching all students enrolled in a specific course

In [7]:
# 1. Fetching all students enrolled in a specific course
course_id = 1  # Replace with the specific course ID
enrolled_students = students_df \
    .filter(F.array_contains(students_df.enrollments.course_id, course_id)) \
    .select("first_name", "last_name", "email")


print("Enrolled Students in Course:", course_id)
enrolled_students.show()



Enrolled Students in Course: 1
+----------+---------+--------------------+
|first_name|last_name|               email|
+----------+---------+--------------------+
|      John|      Doe|john.doe@example.com|
|      Jane|    Smith|jane.smith@exampl...|
+----------+---------+--------------------+



24/09/22 17:15:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 297745 ms exceeds timeout 120000 ms
24/09/22 17:15:26 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/22 17:33:05 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

# 2. Calculating the average number of students enrolled in courses offered by a particular instructor

In [143]:
# Print the schema for both DataFrames
students_df.printSchema()
instructors_df.printSchema()
courses_df.printSchema()
departments_df.printSchema()


root
 |-- _id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- enrollments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- course_id: integer (nullable = true)
 |    |    |-- grade: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)

root
 |-- _id: integer (nullable = true)
 |-- courses_taught: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- department_id: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)

root
 |-- _id: integer (nullable = true)
 |-- course_name: string (nullable = true)
 |-- credits: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- enrollment_count: integer (nullable = true)
 |-- instructors: array (nullable = true)
 |    |-- element: integer (containsNull

In [144]:
# 2. Calculating the average number of students enrolled in courses offered by a particular instructor
instructor_id = 1  # Replace with the specific instructor ID
avg_enrollment = courses_df \
    .filter(F.array_contains(courses_df.instructors, instructor_id)) \
    .agg(F.avg("enrollment_count").alias("average_enrollment"))
    
print("Average Enrollment for Instructor:", instructor_id)
avg_enrollment.show()

Average Enrollment for Instructor: 1
+------------------+
|average_enrollment|
+------------------+
|1.5714285714285714|
+------------------+



# 3. Listing all courses offered by a specific department

In [145]:
# 3. Listing all courses offered by a specific department
department_id = 1  # Replace with the specific department ID
department_courses = courses_df \
    .filter(courses_df.department_id == department_id) \
    .select("course_name")
    
print("Courses in Department:", department_id)
department_courses.show()

Courses in Department: 1
+--------------------+
|         course_name|
+--------------------+
|     Data Structures|
|          Algorithms|
|   Operating Systems|
|Database Manageme...|
|    Network Security|
|    Machine Learning|
+--------------------+



# 4. Finding the total number of students per department

In [146]:
# 4. Finding the total number of students per department
total_students_per_dept = students_df \
    .groupBy("department_id") \
    .agg(F.count("*").alias("total_students"))

print("Total Students per Department:")
total_students_per_dept.show()

Total Students per Department:
+-------------+--------------+
|department_id|total_students|
+-------------+--------------+
|            1|             4|
|            6|             3|
|            3|             4|
|            5|             3|
|            9|             3|
|            4|             3|
|            8|             3|
|            7|             3|
|           10|             3|
|            2|             4|
+-------------+--------------+



# 5. Finding instructors who have taught all the BTech CSE core courses

In [150]:
# Define BTech CSE core courses (assuming these are the first 5 courses in the Computer Science department)
cs_department = departments_df.filter(col("department_name") == "Computer Science").first()
cs_department_id = cs_department["_id"]

core_courses = courses_df.filter(col("department_id") == cs_department_id) \
    .orderBy("_id") \
    .limit(5) \
    .select("_id")

core_course_ids = [row["_id"] for row in core_courses.collect()]

# Find instructors who have taught all core courses
qualified_instructors = instructors_df \
    .filter(size(col("courses_taught")) >= len(core_course_ids)) \
    .filter(array_contains(col("courses_taught"), core_course_ids[0]) &
            array_contains(col("courses_taught"), core_course_ids[1]) &
            array_contains(col("courses_taught"), core_course_ids[2]) &
            array_contains(col("courses_taught"), core_course_ids[3]) &
            array_contains(col("courses_taught"), core_course_ids[4]))

# Display results
print("Instructors who have taught all BTech CSE core courses:")
qualified_instructors.select("first_name", "last_name", "email").show()

Instructors who have taught all BTech CSE core courses:
+----------+---------+--------------------+
|first_name|last_name|               email|
+----------+---------+--------------------+
|      Mark|   Taylor|mark.taylor@examp...|
+----------+---------+--------------------+



In [121]:
instructors_df.printSchema()


root
 |-- _id: integer (nullable = true)
 |-- courses_taught: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- department_id: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)



# 6. Finding top-10 courses with the highest enrollments

In [122]:
# 6. Finding top-10 courses with the highest enrollments
top_courses = courses_df \
    .orderBy(courses_df.enrollment_count.desc()) \
    .limit(10) \
    .select("course_name", "enrollment_count")

print("Top 10 Courses by Enrollment:")
top_courses.show()

Top 10 Courses by Enrollment:
+--------------------+----------------+
|         course_name|enrollment_count|
+--------------------+----------------+
|     Data Structures|               2|
|      Thermodynamics|               2|
|    Circuit Analysis|               2|
|Structural Engine...|               2|
|            Calculus|               2|
|   Operating Systems|               2|
|Database Manageme...|               2|
|    Network Security|               2|
|          Algorithms|               1|
|   Quantum Mechanics|               1|
+--------------------+----------------+



# Stop the Spark session

In [123]:
spark.stop()